<a href="https://colab.research.google.com/github/mapbefun/mapbefun/blob/main/10%EC%A3%BC%EC%B0%A8%EC%B6%9C%EC%84%9D%EA%B3%BC%EC%A0%9C(%EC%98%A4%EC%B0%A8%EC%97%AD%EC%A0%84%ED%8C%8C%EB%B2%95).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

제 5장 오차역전파법_03 & 제 5장 오차역전파법_04를 듣고 아래를 파이썬을 이용하여 구현하고 결과를 분석하시오.<br>
1.함수 softmax, cross_entropy_error를 정의한다.  동영상 03강의 참조<br>
2.클래스 Relu, Sigmoid, Affine, SoftmaxWithLoss를 정의한다. 이들 클래스는 __init__, forward, backward의 함수를 지닌다. 동영상 03강의 참조<br>
3.클래스 ANN을 정의한다. 클래스내에 __init__, predict, accuracy, loss, gradient, update, fit 함수를 정의한다. 동영상 03강의 참조
4.우리가 만든 MNIST 데이터셋의 80%는 훈련용 20%는 검증용으로 만든다.<br>
5.MNIST 데이터셋을 입력받는다. 이때 One_Hot_Encoded data를 입력한다.
6.히든레이어는 1000, 500, 250, 125, 60, 30으로 만들어 학습시킨 후, 검증용 데이터를 이용한 정확도를 계산한다. (우리가 만든 MNIST 검증데이터와 MNIST 검증데이터 각각의 정확도를 계산)<br>
7.회전을 이용하여 데이터를 확장한 후, 다시 학습시킨 후, 검증용 데이터를 이용한 정확도를 계산한다.(우리가 만든 MNIST 검증데이터와 MNIST 검증데이터 각각의 정확도를 계산)<br>
8.확장한 데이터와 MNIST데이터를 통합한 후, 다시 학습시킨 후, 검증용 데이터를 이용한 정확도를 계산한다.(우리가 만든 MNIST 검증데이터와 MNIST 검증데이터 각각의 정확도를 계산)<br>
9.위를 Keras로 구현한다.<br>

In [ ]:
def cross_entropy_error(y,t):  #크로스엔트로피 함수를 정의한다. x값을 넣으면 Prediction해야 하므로 Prediction 된 y값을 넣는다. 
  if y.size == t.size:
    t = np.argmax(t,axis=1)  #가로 축에서 어느 값이 최대값을 갖는지를 보기 위해 argmax함수를 쓴다. 

  n_size = y.shape[0]   #데이터의 사이즈를 정의: 0번째 데이터가 몇 개들어 왔는지

  return -np.sum(np.log(y[np.arange(n_size),t]+1e-7))/n_size 

In [ ]:
#분류하기 위해 소프트맥스 함수를 정의한다(이차원으로)
def softmax(a):
    if a.ndim == 2:
        a=a.T
        _max=np.max(a,axis=0)
        exp_a=np.exp(a-_max) #a에서 max값을 빼준다
        y=exp_a/np.sum(exp_a,axis=0)
        return y.T
    else: 
            _max=np.max(a)
            exp_a=np.exp(a-_max)
            y=exp_a/np.sum(exp_a)
            return y  

In [ ]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss=None
        self.y=None
        self.t=None

    def forward(self,x,t):
        self.t=t
        self.y=softmax(x)
        self.loss=cross_entropy_error(self.t)
        return self.loss

    def backward(self,dout):
        dx=(self.y-self.t)/self.t.shape[0]
        return dx



In [ ]:

# 2층 신경망
# __init__ --> 파라미터를 초기화
# predict  --> 추론
# accuracy --> 정확도
# loss     --> 손실함수
# gradient --> 경사도 방향벡터
# update   --> 파라미터 갱신
# fit      --> 학습 데이터..  -> gradient -> update -> gradient -> update... (학습)

from collections import OderedDict

class ANN:
  def __init__(self,input_size,hidden_size,output_size,learning_rate,batch_size):

    self.lr = learning_rate
    self.batch_size = batch_size

    self.param = {}
    self.grad = {}

    if not isinstance(hidden_size,list):
      hidden_size = [hidden_size]

    nNeuron = [input_size] + hidden_size + [output_size]
    self.nLayer = len(nNeuron)-1

    self.layers = OderedDict  # (딕셔너리가 있는) 연산레이어 생성


    for i in range(self.nLayer):
      self.param['W'+str(i+1)] = np.sqrt(2/nNeuron[i])*np.random.randn(nNeuron[i],nNeuron[i+1])
      self.param['b'+str(i+1)] = np.zeros(nNeuron[i+1])
      self.layers['Affine'+Str(i+1)]=Affine(self.param['W'+str(i+1)],self.parm['b'+str(i+1)])

      if i !=self.nLayer -1:
          self.layer['relu'+str(i+1)] = Relu()
      else:
          self.lastLayer = SofmaxLoss()


  def predict(self,x):
   for layer in self.layer.values():
       x=layer.forward(x)
   return x     

# accuracy함수는 검증을 위한 함수임

  def accuracy(self,x,t):
    batch_size = self.batch_size
    acc = 0
    for i in np.arane(0,x.shape[0],batch_size):
      y = self.predict(x[i:i+batch_size])

      if y.size == t[i:i+batch_size].size:
        pt = np.argmax(t[i:i+batch_size],axis=1)
      else:
        pt = t[i:i+batch_size]

      p = np.argmax(y,axis=1)

      acc+= np.sum(p==pt)

    return axx/x.shape[0]


  def loss(self,x,t):
    y = self.predict(x)
    return self.lastLayer.forward(y,t)

  def gradient(self,x,t):
    _loss = self.loss(x,t)

    #backward
    dout = 1 
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layers.reverse()
    for layer in layers:
        dout = layer.backward(dout)
    return _loss 


  def update(self):
    for i in range(self.nLayer):
      self.layers['Affine'+str(i+1).W -= self.lr*self.layers['Affine'+str(i+1)].dW
      self.param['Affine'+str(i+1).b -= self.lr*self.layers['Affine'+str(i+1)].dW


  def fit(self,xTrain,tTrain,xTest,tTest,epoch):

    for i in range(epoch):
      vec = np.arange(xTrain.shape[0])
      np.random.shuffle(vec)
      xTrain = xTrain[vec]
      tTrain = tTrain[vec]
    
      nBatch = xTrain.shape[0]//self.batch_size

      for j in range(nBatch):
        x = xTrain[j*self.batch_size:(j+1)*self.batch_size]
        t = tTrain[j*self.batch_size:(j+1)*self.batch_size]


        #gradient 구하기
        _loss = self.gradient(x,t)
        #update 하기
        self.update()
      
        print(f"{j}번째 업데이트: 손실값 {_loss}")

    print("")
    print(f"{i}번째 epoch LOSS: {_loss}, 정확도: {self.accuracy(xTest,tTest)}")
    


In [ ]:
from google.colab import drive
drive.mount('/datalab/drive')

In [ ]:
import sys, os
sys.path.append(os.pardir)
sys.path.append('/datalab/drive/MyDrive/딥러닝수업')
my_path = '/datalab/drive/MyDrive/딥러닝수업/'

In [ ]:
import pickle

with open(my_path+"OurMNIST/OurMNISTdata.pkl" ,"rb") as f:
  our_images=pickle.load(f)
with open(my_path+"OurMNIST/OurMNISTlabel.pkl" ,"rb") as f:
  our_labels=pickle.load(f)

print(our_images.shape)
print(our_labels.shape)

In [ ]:
# one_hot_encoding
def one_hot_encoder(data,nclass):
    rows = np.zeros((data.shape[0]))
    one_hot = np.zeros((data.shape[0],nclass),dtype="int32")
    one_hot[rows,data]=1
    return one_hot

In [ ]:
our_labels = one_hot_encoder(our_labels,10)


In [ ]:
out_images_1d = our_images.reshape(-1,28*28)
pos = []
for i,img in enumerate(our_images_1d):
    if np.sum(np.nan(img)):